##### Pororo를 활용한 역번역
- train_dataset에 존재하는 question 데이터를 증강하기 위해서 사용했습니다.
- question을 제외한 기존의 필드는 유지하는 조건을 가져갑니다.

----------
##### import Pororo

In [1]:
import pororo
from pororo import Pororo

/data/ephemeral/home/shark/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


----------
##### 필요 Import

In [42]:
import nltk
import torch
import random
from datasets import load_from_disk, Dataset, DatasetDict, concatenate_datasets


----------
##### 초기 설정
- nltk.download 진행 시 'punkt'가 아닌 'punkt_tab'을 입력  
※ 'punkt' 입력할 경우 mt(mt(text, src="ko", tgt="en"), src="en", tgt="ko")에서 오류 발생

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
nltk.download('punkt_tab')
mt = Pororo(task="translation", lang="multi", model='transformer.large.multi.mtpg')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


----------
##### 데이터 불러오기

In [15]:
disk = "/data/ephemeral/home/level2-mrc-nlp-12/data"

train_data = load_from_disk(disk + "/train_dataset/")
train_dataset = train_data["train"]
print(len(train_data['train']))

3952


----------
##### 역번역

In [28]:
def back_translation(text):
    return mt(mt(text, src="ko", tgt="en"), src="en", tgt="ko")

In [29]:
text = "케빈은 집에 갔다. 그러나 더 놀고 싶어한다."
test = back_translation(text)
print(test)

케빈은 집에 갔다 하지만 나는 더 뛰고 싶다


----------
##### 증강

In [30]:
def augment_data(train_dataset, limit):
    augmented_examples = []
    for idx, example in enumerate(train_dataset):
        if idx >= limit:
            break
        original_question = example['question']
        
        #역번역
        back_translated_question = back_translation(original_question)
        
        # 새로운 예제 생성
        new_example = example.copy()
        new_example['question'] = back_translated_question
        
        augmented_examples.append(new_example)
    
    # 리스트를 딕셔너리로 변환하여 Dataset 생성
    aug_train_data_dict = {key: [ex[key] for ex in augmented_examples] for key in augmented_examples[0]}
    aug_train_data = Dataset.from_dict(aug_train_data_dict)
    
    return aug_train_data

In [34]:
#제한 개수
augmentation_limit = len(train_data['train'])       #전체

aug_train_data = augment_data(train_dataset, augmentation_limit)

----------
##### 데이터 출력 (상위 10개)

In [51]:
for idx, (original, augmented) in enumerate(zip(train_dataset, aug_train_data)):
    if idx >= 10:
        break
    print("[", idx, "]" + f" 기존 질문  : {original['question']}")
    print("[", idx, "]" + f" 역번역 질문: {augmented['question']}\n")

[ 0 ] 기존 질문  : 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
[ 0 ] 역번역 질문: 어떤 국가기관이 대통령을 포함한 미국 행정부를 견제할 권리가 있는가?

[ 1 ] 기존 질문  : 현대적 인사조직관리의 시발점이 된 책은?
[ 1 ] 역번역 질문: 현대 인사 운영의 출발점이 어떤 책이 됐을까?

[ 2 ] 기존 질문  : 강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
[ 2 ] 역번역 질문: 강희제가 누구를 위해 1717년에 썼나요?

[ 3 ] 기존 질문  : 11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?
[ 3 ] 역번역 질문: 보통 11세기와 12세기에 전국의 특성이 어떤 것이냐는 것이다.

[ 4 ] 기존 질문  : 명문이 적힌 유물을 구성하는 그릇의 총 개수는?
[ 4 ] 역번역 질문: 그 이름으로 작성된 유물의 전체 병은 몇 개입니까?

[ 5 ] 기존 질문  : 카드모스의 부하들이 간 곳에는 무엇이 있었는가?
[ 5 ] 역번역 질문: 카드모스 남자들이 어디로 갔지?

[ 6 ] 기존 질문  : 관우를 불태워 죽이려한 사람 누구인가?
[ 6 ] 역번역 질문: 관을 태우려던 사람은 누구입니까?

[ 7 ] 기존 질문  : 참호 속에 무기와 장비를 버리고 도주한 집단은?
[ 7 ] 역번역 질문: 재난 속에서 무기와 장비를 버린 사람들이 한 무리인가요?

[ 8 ] 기존 질문  : 제2차 세계 대전에 참전하여 사망한 자식은?
[ 8 ] 역번역 질문: 2차 세계대전에서 누가 죽었나요?

[ 9 ] 기존 질문  : 고려 공민왕이 처가 식구들과 아내와 함께 피신처로 삼은 마을은?
[ 9 ] 역번역 질문: 고려왕은 어떤 마을에서 아내의 가족과 아내를 피난처로 만들었을까?



##### 랜덤한 30개 출력

In [53]:
random_indices = random.sample(range(len(train_dataset)), 30)

# 원래 질문과 역번역된 질문 출력 (랜덤 30개)
for idx in random_indices:
    original = train_dataset[idx]
    augmented = aug_train_data[idx]
    print("[", idx, "]" + f" 기존 질문  : {original['question']}")
    print("[", idx, "]" + f" 역번역 질문: {augmented['question']}\n")

[ 1634 ] 기존 질문  : 두 번째, 세 번째 피아노 협주곡 중 먼저 세상에 알려진 곡은?
[ 1634 ] 역번역 질문: 두 번째와 세 번째 피아노 콘서트가 세계에서 처음 알려진 곡인가?

[ 2082 ] 기존 질문  : 미국과 달리 이민자 남성과 현지인의 결혼이 가능했던 국가는?
[ 2082 ] 역번역 질문: 미국과 달리 이민자와 현지인들이 결혼할 수 있는 곳은 어디인가?

[ 140 ] 기존 질문  : 파이어폭스 이전 가장 잘나가던 웹 브라우저는?
[ 140 ] 역번역 질문: 불폭스 역사상 최고의 웹 브라우저는 무엇입니까?

[ 3884 ] 기존 질문  : 프랑수아 뒤발리에를 뒤를 이어 대통령이 된 자는?
[ 3884 ] 역번역 질문: 판타지아의 꼬리를 물고 누가 대통령이 되었나요?

[ 448 ] 기존 질문  : 국문과 역사를 가르치던 곳에서 민족의지의 격려를 위해 불렀던 노래는 무엇인가?
[ 448 ] 역번역 질문: 한국 문화와 역사를 가르치는 자리에서 국민들의 뜻을 부추기기 위해 부른 노래는 무엇이었을까.

[ 691 ] 기존 질문  : 예측할 수 없는 대류의 움직임 때문에 발생하게 되는 것은?
[ 691 ] 역번역 질문: 분위기의 예측 불가능한 움직임 때문에 어떻게 될까?

[ 3205 ] 기존 질문  : 네팔의 헌법 수립안 투표에서 기권한 사람의 수는?
[ 3205 ] 역번역 질문: 네팔 헌법 수립에 투표한 사람들은 몇 명이나 되죠?

[ 1091 ] 기존 질문  : 윤치오가 특파대사직에 고용된 것을 막으려한 나라는?
[ 1091 ] 역번역 질문: 윤치오 특사가 특사로 채용되는 것을 막으려는 나라가 있는가?

[ 1097 ] 기존 질문  : 티무르 다음으로 바그다드를 다스리게 된 사람은?
[ 1097 ] 역번역 질문: 바그다드를 지배하는 팀우 옆에는 누가 될까?

[ 1970 ] 기존 질문  : 메디치 가문의 몰락 후 토스카나를 넘겨받은 사람은?
[ 1970 ] 역번역 질문: 누가 메디카니 가족이 추락한 뒤 도스카나를 인수했을까?

[ 381

----------
##### 새로운 disk에 저장

In [43]:
combined_data = DatasetDict({'train': concatenate_datasets([train_dataset, aug_train_data])})
combined_data.save_to_disk(disk + "/pororoBT_en_train_data/")

Saving the dataset (1/1 shards): 100%|██████████| 7904/7904 [00:00<00:00, 219420.33 examples/s]


In [48]:
combined_data['train'][random(len(combined_data))]

TypeError: 'module' object is not callable